In [1]:
!ls '/content/drive/My Drive/Application Tracking System - Neural Networks/*.py'

ls: cannot access '/content/drive/My Drive/Application Tracking System - Neural Networks/*.py': No such file or directory


In [2]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 9904743610955786113, name: "/device:XLA_CPU:0"
 device_type: "XLA_CPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 735960473595549761
 physical_device_desc: "device: XLA_CPU device"]

In [3]:
!pip install keras

In [4]:
cd /content/drive/My Drive/Application Tracking System - Neural Networks

/content/drive/My Drive/Application Tracking System - Neural Networks


In [5]:
!cat '/content/drive/My Drive/Application Tracking System - Neural Networks/util.py'

import re

from tensorflow.python.keras import backend as K
from tensorflow.python.keras.layers import Layer
from tensorflow.python.keras.preprocessing.sequence import pad_sequences

from nltk.corpus import stopwords
from gensim.models import KeyedVectors

import gensim

import numpy as np

import itertools

EMBEDDING_FILE = './data/Resume_JD_new_13.w2v'

def text_to_word_list(text):
    # Pre process and convert texts to a list of words
    text = str(text)
    text = text.lower()

    # Clean the text
    text = re.sub(r"[^A-Za-z0-9^,!.\/'+-=]", " ", text)
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "cannot ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r",", " ", text)
    text = re.sub(

In [6]:
#load all packages
import re

from tensorflow.python.keras import backend as K
from tensorflow.python.keras.layers import Layer
from tensorflow.python.keras.preprocessing.sequence import pad_sequences

from nltk.corpus import stopwords
from gensim.models import KeyedVectors

import gensim

import numpy as np

import itertools

In [7]:
def text_to_word_list(text):
    # Pre process and convert texts to a list of words
    text = str(text)
    text = text.lower()

    # Clean the text
    text = re.sub(r"[^A-Za-z0-9^,!.\/'+-=]", " ", text)
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "cannot ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r",", " ", text)
    text = re.sub(r"\.", " ", text)
    text = re.sub(r"!", " ! ", text)
    text = re.sub(r"\/", " ", text)
    text = re.sub(r"\^", " ^ ", text)
    text = re.sub(r"\+", " + ", text)
    text = re.sub(r"\-", " - ", text)
    text = re.sub(r"\=", " = ", text)
    text = re.sub(r"'", " ", text)
    text = re.sub(r"(\d+)(k)", r"\g<1>000", text)
    text = re.sub(r":", " : ", text)
    text = re.sub(r" e g ", " eg ", text)
    text = re.sub(r" b g ", " bg ", text)
    text = re.sub(r" u s ", " american ", text)
    text = re.sub(r"\0s", "0", text)
    text = re.sub(r" 9 11 ", "911", text)
    text = re.sub(r"e - mail", "email", text)
    text = re.sub(r"j k", "jk", text)
    text = re.sub(r"\s{2,}", " ", text)

    text = text.split()

    return text

In [28]:
EMBEDDING_FILE = './data/Resume_JD_new_13.w2v'
#word2vec embedding
def make_w2v_embeddings(df, embedding_dim=300, empty_w2v=False):
    vocabs = {}
    vocabs_cnt = 0

    vocabs_not_w2v = {}
    vocabs_not_w2v_cnt = 0

    # Stopwords
    stops = set(stopwords.words('english'))

    # Load word2vec
    print("Loading word2vec model(it may takes 2-3 mins) ...")

    if empty_w2v:
        word2vec = EmptyWord2Vec
    else:
        word2vec = KeyedVectors.load_word2vec_format(EMBEDDING_FILE, binary=True)
                    
    for index, row in df.iterrows():
        # Print the number of embedded sentences.
        if index != 0 and index % 1000 == 0:
            print("{:,} sentences embedded.".format(index), flush=True)

        # Iterate through the text of both questions of the row
        for question in ['Resume', 'JD']:

            q2n = []  # q2n -> question numbers representation
            for word in text_to_word_list(row[question]):
                # Check for unwanted words
                if word in stops:
                    continue

                # If a word is missing from word2vec model.
                if word not in word2vec.vocab:
                    if word not in vocabs_not_w2v:
                        vocabs_not_w2v_cnt += 1
                        vocabs_not_w2v[word] = 1

                # If you have never seen a word, append it to vocab dictionary.
                if word not in vocabs:
                    vocabs_cnt += 1
                    vocabs[word] = vocabs_cnt
                    q2n.append(vocabs_cnt)
                else:
                    q2n.append(vocabs[word])

            # Append question as number representation
            df.at[index, question + '_n'] = q2n

    embeddings = 1 * np.random.randn(len(vocabs) + 1, embedding_dim)  # This will be the embedding matrix
    embeddings[0] = 0  # So that the padding will be ignored

    # Build the embedding matrix
    for word, index in vocabs.items():
        if word in word2vec.vocab:
            embeddings[index] = word2vec.word_vec(word)
    del word2vec

    return df, embeddings

#zero padding
def split_and_zero_padding(df, max_seq_length):
    # Split to dicts
    X = {'left': df['Resume_n'], 'right': df['JD_n']}

    # Zero padding
    for dataset, side in itertools.product([X], ['left', 'right']):
        dataset[side] = pad_sequences(dataset[side], padding='pre', truncating='post', maxlen=max_seq_length)

    return dataset


#  --
#Manhattan distance
class ManDist(Layer):
    """
    Keras Custom Layer that calculates Manhattan Distance.
    """

    # initialize the layer, No need to include inputs parameter!
    def __init__(self, **kwargs):
        self.result = None
        super(ManDist, self).__init__(**kwargs)

    # input_shape will automatic collect input shapes to build layer
    def build(self, input_shape):
        super(ManDist, self).build(input_shape)

    # This is where the layer's logic lives.
    def call(self, x, **kwargs):
        self.result = K.exp(-K.sum(K.abs(x[0] - x[1]), axis=1, keepdims=True))
        return self.result

    # return output shape
    def compute_output_shape(self, input_shape):
        return K.int_shape(self.result)


class EmptyWord2Vec:
    """
    Just for test use.
    """
    vocab = {}
    word_vec = {}

In [9]:
import pandas as pd

import tensorflow as tf

from util import make_w2v_embeddings
from util import split_and_zero_padding
from util import ManDist

In [ ]:
from pymongo import MongoClient

client = MongoClient('localhost:27017')
db = client.Applications
collection = db.sample_1

# print("Enter Resume File pathname")
# for filepath in list(glob.iglob(input())):
#     if filepath.endswith(".jpg") or filepath.endswith(".png"):
#         img = Image.open(filepath)
#         text1 = terr.image_to_string(img)
#         text = text1.lower()
#         tx = " ".join(text.split('\n'))
#         print(tx)
#         collection.insert_one(tx)
#     elif filepath.endswith(".pdf") or filepath.endswith(".docx"):
#         data = ResumeParser(filepath).get_extracted_data()
#         data_mongo = data
#         collection.insert_one(data_mongo)
num_doc = collection.find().count()
# print(f'Number of documents inserted in mongodb are:{num_doc}')
resume_skills_work = collection.find({},{'_id': 0, 'skills':1,  
                 'experience': 1})
test_df1 = pd.DataFrame(list(resume_skills_work))
test_df1['Resume'] = test_df1['skills'] + test_df1['experience']
test_df1.dropna()

In [30]:

TEST_CSV = '/content/drive/My Drive/Application Tracking System - Neural Networks/MLSTM_TEST_1.xlsx'
test_df3 = pd.read_excel(TEST_CSV)
for q in ['Resume', 'JD']:
    test_df3[q + '_n'] = test_df3[q]
test_df3

,Resume,JD,Score,Resume_n,JD_n
0,Worked with advance web development technologi...,Worked with advance web development technologi...,5,Worked with advance web development technologi...,Worked with advance web development technologi...
1,Maximized use of Events and promises in ES6 an...,Maximized use of Events and promises in ES6 an...,5,Maximized use of Events and promises in ES6 an...,Maximized use of Events and promises in ES6 an...
2,Developed Drag-gable and Re-sizable modals usi...,Developed Drag-gable and Re-sizable modals usi...,5,Developed Drag-gable and Re-sizable modals usi...,Developed Drag-gable and Re-sizable modals usi...
3,Enforced Agile and Scrum development methodolo...,Enforced Agile and Scrum development methodolo...,5,Enforced Agile and Scrum development methodolo...,Enforced Agile and Scrum development methodolo...
4,"Developed sound knowledge on React, Redux for ...","Developed sound knowledge on React, Redux for ...",4,"Developed sound knowledge on React, Redux for ...","Developed sound knowledge on React, Redux for ..."
5,Worked with advance web development technologi...,"Strong experience in JavaScript, ideally ES6 o...",5,Worked with advance web development technologi...,"Strong experience in JavaScript, ideally ES6 o..."
6,Validated customer interface requirements and ...,4+ years of experience with customer facing we...,5,Validated customer interface requirements and ...,4+ years of experience with customer facing we...
7,"Updated versions of React, React-Router.",Experience in building single page application...,5,"Updated versions of React, React-Router.",Experience in building single page application...
8,Introduced Ag-Grid package which can plot mill...,NaN,5,Introduced Ag-Grid package which can plot mill...,NaN
9,Worked with advance web development technologi...,Familiarity with ES6 and newer versions of Jav...,5,Worked with advance web development technologi...,Familiarity with ES6 and newer versions of Jav...


In [31]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [32]:
use_w2v = True

In [33]:
embedding_dim = 300
max_seq_length = 70
test_df3, embeddings = make_w2v_embeddings(test_df3, embedding_dim=embedding_dim, empty_w2v=use_w2v)

# Split to dicts and append zero padding.
X_test = split_and_zero_padding(test_df3, max_seq_length)

# Make sure everything is ok
assert X_test['left'].shape == X_test['right'].shape

Loading word2vec model(it may takes 2-3 mins) ...


In [34]:
model = tf.keras.models.load_model('./data/SiameseLSTM_new20.h5', custom_objects={'ManDist': ManDist})
model.summary()

Model: "functional_5"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_9 (InputLayer)            [(None, 70)]         0                                            
__________________________________________________________________________________________________
input_10 (InputLayer)           [(None, 70)]         0                                            
__________________________________________________________________________________________________
sequential_5 (Sequential)       (None, 64)           2111520     input_9[0][0]                    
                                                                 input_10[0][0]                   
__________________________________________________________________________________________________
man_dist_5 (ManDist)            (None, 1)            0           sequential_5[0][0]    

In [62]:
prediction = model.predict([X_test['left'], X_test['right']]) * 100
prediction
for i in prediction:
    if 90 <= i <= 100:
        print("Score:",5)
    elif 70 <= i < 90:
        print("Score:", 4)
    elif 55 <= i < 70:
        print("Score:", 3)
    elif 40 <= i <55:
        print("Score:", 2)
    else:
        print("Score:", 1)

Score: 5
Score: 5
Score: 5
Score: 5
Score: 5
Score: 1
Score: 1
Score: 1
Score: 1
Score: 1
Score: 1
Score: 1
Score: 1
Score: 1
Score: 1
Score: 1
Score: 1
Score: 1
Score: 2
Score: 1
Score: 1
Score: 1
Score: 1
